In [1]:
from pyspark.sql import SparkSession

In [2]:
from pyspark.sql import functions as f
from pyspark.sql.types import LongType

In [3]:
spark = SparkSession.builder.config('spark.sql.adaptive.enabled','True').getOrCreate()

In [4]:
codes_df = spark.read.csv('airport-codes-na.txt', inferSchema=True, header=True, sep='\t')

In [5]:
dd_df = spark.read.csv('departuredelays.csv', inferSchema=True, header=True)

In [6]:
dd_df.dtypes

[('date', 'int'),
 ('delay', 'int'),
 ('distance', 'int'),
 ('origin', 'string'),
 ('destination', 'string')]

In [7]:
dd_df = dd_df.withColumn('delay', f.col('delay').cast('int'))
dd_df = dd_df.withColumn('distance', f.col('distance').cast('int'))

In [8]:
# dd_df.withColumn('delay',f.expr('CAST(delay AS int) as delay')).show()

In [9]:
sdd_df = dd_df.filter(f.expr("""origin=='SEA' AND destination=='SFO' AND delay > 0 and date like '1010%' """))
sdd_df.show()
# sdd_df = dd_df.where((f.col('origin') == 'SEA') & (f.col('destination')=='SFO') & f.col('date').like('1010%'))
# sdd_df.show()

+-------+-----+--------+------+-----------+
|   date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
|1010710|   31|     590|   SEA|        SFO|
|1010955|  104|     590|   SEA|        SFO|
|1010730|    5|     590|   SEA|        SFO|
+-------+-----+--------+------+-----------+



In [10]:
sdd2_df= sdd_df.union(sdd_df)

In [11]:
sdd2_df.show()

+-------+-----+--------+------+-----------+
|   date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
|1010710|   31|     590|   SEA|        SFO|
|1010955|  104|     590|   SEA|        SFO|
|1010730|    5|     590|   SEA|        SFO|
|1010710|   31|     590|   SEA|        SFO|
|1010955|  104|     590|   SEA|        SFO|
|1010730|    5|     590|   SEA|        SFO|
+-------+-----+--------+------+-----------+



In [12]:
sdd_extra = sdd_df.withColumn('sdd_x2',sdd_df['delay']*2)

In [13]:
# Fails when # cols!=in each table
# test_df = sdd_extra.union(sdd_df)
# test_df

# What about differing col names? Same order
sdd_df.union(sdd2_df.withColumnRenamed("del2",'delay')).show()

+-------+-----+--------+------+-----------+
|   date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
|1010710|   31|     590|   SEA|        SFO|
|1010955|  104|     590|   SEA|        SFO|
|1010730|    5|     590|   SEA|        SFO|
|1010710|   31|     590|   SEA|        SFO|
|1010955|  104|     590|   SEA|        SFO|
|1010730|    5|     590|   SEA|        SFO|
|1010710|   31|     590|   SEA|        SFO|
|1010955|  104|     590|   SEA|        SFO|
|1010730|    5|     590|   SEA|        SFO|
+-------+-----+--------+------+-----------+



In [14]:
codes_df.dtypes

[('City', 'string'),
 ('State', 'string'),
 ('Country', 'string'),
 ('IATA', 'string')]

In [15]:
j_df = codes_df.join(sdd_df, on=codes_df['IATA']==sdd_df['origin'])

In [16]:
j_df.show()

+-------+-----+-------+----+-------+-----+--------+------+-----------+
|   City|State|Country|IATA|   date|delay|distance|origin|destination|
+-------+-----+-------+----+-------+-----+--------+------+-----------+
|Seattle|   WA|    USA| SEA|1010710|   31|     590|   SEA|        SFO|
|Seattle|   WA|    USA| SEA|1010955|  104|     590|   SEA|        SFO|
|Seattle|   WA|    USA| SEA|1010730|    5|     590|   SEA|        SFO|
+-------+-----+-------+----+-------+-----+--------+------+-----------+



In [17]:
j_df.groupby('origin').pivot('origin').avg().show()

+------+-----------------------------+------------------------------+---------------------------------+
|origin|SEA_avg(CAST(date AS BIGINT))|SEA_avg(CAST(delay AS BIGINT))|SEA_avg(CAST(distance AS BIGINT))|
+------+-----------------------------+------------------------------+---------------------------------+
|   SEA|           1010798.3333333334|            46.666666666666664|                            590.0|
+------+-----------------------------+------------------------------+---------------------------------+



In [18]:
j_df.groupby('origin','destination').agg(f.isnan('origin')).show()

+------+-----------+-------------+
|origin|destination|isnan(origin)|
+------+-----------+-------------+
|   SEA|        SFO|        false|
+------+-----------+-------------+



In [19]:
j_df.write.parquet('test.parquet', mode='overwrite')
t_df = spark.read.parquet('test.parquet')

In [20]:
t_df.show()

+-------+-----+-------+----+-------+-----+--------+------+-----------+
|   City|State|Country|IATA|   date|delay|distance|origin|destination|
+-------+-----+-------+----+-------+-----+--------+------+-----------+
|Seattle|   WA|    USA| SEA|1010710|   31|     590|   SEA|        SFO|
|Seattle|   WA|    USA| SEA|1010955|  104|     590|   SEA|        SFO|
|Seattle|   WA|    USA| SEA|1010730|    5|     590|   SEA|        SFO|
+-------+-----+-------+----+-------+-----+--------+------+-----------+

